In [4]:
# Import the relevant libraries
import pandas as pd
import time
from datetime import datetime, timedelta
import os
import folium
import csv

In [5]:
# working on a one file data
folder="./data/crowdsource-internal/"

msisdn ='6588769918'
log ='6588769918_20170210_210100.log'#for exploring only



In [8]:
f = os.path.join(folder,log)
type(f)

str